## DENSENET

In [1]:
import d2l 
from mxnet import np, npx
from mxnet.gluon import nn
npx.set_np()

In [2]:
def conv_block(num_channels):
    blk = nn.Sequential()
    blk.add(nn.BatchNorm(),
           nn.Activation('relu'),
           nn.Conv2D(num_channels, kernel_size=3, padding=1))
    return blk

In [4]:
class DenseBlock(nn.Block):
    def __init__(self, num_convs, num_channels, **kwargs):
        super(DenseBlock, self).__init__(**kwargs)
            
        self.net = nn.Sequential()
        
        for _ in range(num_convs):
            self.net.add(conv_block(num_channels))
            
    def forward(self, X):
        
        for blk in self.net:
            Y = blk(X)
            X = np.concatenate((X,Y), axis=1)
        return X

In [5]:
blk = DenseBlock(2,10)
blk.initialize()
X = np.random.uniform(size=(4,3,8,8))
Y = blk(X)
Y.shape

(4, 23, 8, 8)

In [6]:
def transition_block(num_channels):
    blk = nn.Sequential()
    blk.add(nn.BatchNorm(), nn.Activation('relu'),
           nn.Conv2D(num_channels, kernel_size=1),
           nn.AvgPool2D(pool_size=2, strides=2))
    return blk

In [7]:
blk = transition_block(10)
blk.initialize()
blk(Y).shape

(4, 10, 4, 4)

In [8]:
net = nn.Sequential()
net.add(nn.Conv2D(64, kernel_size=7,strides=2, padding=3),
       nn.BatchNorm(), nn.Activation('relu'),
       nn.MaxPool2D(pool_size=3, strides=2, padding=1))

In [9]:
num_channels, growth_rate = 64,32
num_convs_in_dense_blocks = [4,4,4,4]

for i, num_convs in enumerate(num_convs_in_dense_blocks):
    net.add(DenseBlock(num_convs, growth_rate))
    num_channels += num_convs * growth_rate
    
    if i != len(num_convs_in_dense_blocks) -1:
        num_channels //= 2
        net.add(transition_block(num_channels))

In [10]:
net.add(nn.BatchNorm(),
       nn.Activation('relu'),
       nn.GlobalAvgPool2D(),
       nn.Dense(10))

In [ ]:
lr, num_epochs, batch_size = 0.1, 1, 256
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size, resize=96)
d2l.train_ch6(net, train_iter, test_iter, num_epochs, lr)